<a href="https://colab.research.google.com/github/TasfiaTahsinAnnita/CSE438/blob/main/Assignment_PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt_dlp
!pip install opencv-python-headless --quiet

In [ ]:
import yt_dlp
import cv2
import numpy as np
import urllib.request
import os
from IPython.display import display, Video

# **Download Video from YouTube**

The script starts by defining a function to download a video from YouTube using the yt-dlp library. This function takes a video URL and the path where the video should be saved locally.

yt-dlp.YoutubeDL(): Initializes the downloader with options to save the video to the given path and choose the best available quality.

ydl.download([video_url]): Downloads the video from the provided URL.

In [ ]:
def download_youtube_video(video_url, output_path):
    ydl_opts = {
        'outtmpl': output_path,
        'format': 'best',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

# **Check if Video Exists or Download**

Next, the script defines the YouTube video URL and checks whether the video already exists locally. If the video doesn't exist, it will be downloaded using the download_youtube_video function defined above.

VIDEO_PATH: This is the path where the video will be saved (in this case, downloaded_video.mp4).

os.path.exists(): Checks if the video already exists at the specified path. If it doesn't, the script proceeds to download the video.

In [ ]:
video_url = "https://youtu.be/RJ60ON-GSo8?si=UcRD_5Wc9OsS00b9"
VIDEO_PATH = "downloaded_video.mp4"

# verifying download
if not os.path.exists(VIDEO_PATH):
    download_youtube_video(video_url, VIDEO_PATH)

# **Open Video Using OpenCV**
Once the video is downloaded (or if it already exists), the script opens the video using OpenCV's cv2.VideoCapture() method. It then retrieves the video's frame width, height, and FPS.

cv2.VideoCapture(): Opens the video file.

cap.get(): Retrieves properties like the video’s frame dimensions and frames per second (FPS).

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# **Initialize Video Output Writers**
The script creates two video writers using OpenCV to save processed videos:

Global Histogram Equalization (HE) Output.

CLAHE Comparison Output.

cv2.VideoWriter(): Initializes the video writers. The first argument is the output file name, the second argument defines the codec, the third argument is the FPS, and the fourth argument defines the output frame size.

In [ ]:
out = cv2.VideoWriter('outputyt_hist_eq_grid.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps if fps > 0 else 24,
                      (frame_width * 2, frame_height * 2))

clahe_out = cv2.VideoWriter('outputyt_clahe_comparison.mp4',
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            fps if fps > 0 else 24,
                            (frame_width * 2, frame_height))



## **Histogram**
----
The histogram of gray level digital image with intensity values between $[0, L - 1]$ is a discrete function:

<font size="5">
$$h(r_k) = n_k$$
</font>

* $r_k$ : $k$-th intensity value
* $n_k$ : the number of pixels with $r_k$ value in the image




In [ ]:
def make_histogram_panel(bgr_frame, size=(320, 240), bins=64):
    panel = np.zeros((size[1], size[0], 3), dtype=np.uint8)
    colors = ('b', 'g', 'r')
    for i, color in enumerate(colors):
        hist = cv2.calcHist([bgr_frame], [i], None, [bins], [0, 256])
        cv2.normalize(hist, hist, alpha=0, beta=size[1], norm_type=cv2.NORM_MINMAX)
        bin_width = int(size[0] / bins)
        for j in range(1, bins):
            cv2.line(panel,
                     (bin_width*(j-1), size[1] - int(hist[j-1])),
                     (bin_width*j, size[1] - int(hist[j])),
                     color=(255 if color == 'b' else 0,
                            255 if color == 'g' else 0,
                            255 if color == 'r' else 0),
                     thickness=2)
    return panel


# **RMS Contrast Function**

The Root Mean Square (RMS) contrast function calculates the contrast of a grayscale image. It is defined as the standard deviation of pixel intensities.

The Root Mean Square (RMS) Contrast is a measure of image contrast, calculated as:

image.png

Where:

𝐼𝑖 is the pixel intensity at position
𝑖,

𝜇 is the mean pixel intensity,

𝑁 is the total number of pixels in the image.

This function calculates the square root of the average squared deviation from the mean of pixel values.

In [ ]:
def rms_contrast(gray_img):
    return np.sqrt(np.mean(np.square(gray_img - np.mean(gray_img))))

# **Main Processing Loop**

The main loop processes the video frame by frame:

Global Histogram Equalization (HE): Enhances the global contrast of the image.

CLAHE (Contrast Limited Adaptive Histogram Equalization): Enhances local contrast.

Global Histogram Equalization: Uses cv2.equalizeHist() to enhance the contrast of the Y-channel (luminance) in the YCrCb color space.

CLAHE: Applies adaptive histogram equalization using cv2.createCLAHE().

# **Writing Output Videos**

After processing each frame, the script:

Creates a 2x2 grid of the original and processed frames along with their histograms.

Saves the 2x2 grid and the CLAHE comparison side by side.

np.hstack(): Stacks images horizontally.

np.vstack(): Stacks images vertically.

The resulting frames are written to the video files using out.write() and clahe_out.write().

# **Print RMS Contrast**
For each frame, the RMS contrast of the processed frames (Global HE and CLAHE) is printed to the console.

This helps to track the contrast changes between the original and processed frames.

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original = frame.copy()

    # Global Histogram Equalisation
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    y_eq = cv2.equalizeHist(ycrcb[:, :, 0])
    ycrcb[:, :, 0] = y_eq
    he_frame = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    # CLAHE
    ycrcb_clahe = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    y_clahe = clahe.apply(ycrcb_clahe[:, :, 0])
    ycrcb_clahe[:, :, 0] = y_clahe
    clahe_frame = cv2.cvtColor(ycrcb_clahe, cv2.COLOR_YCrCb2BGR)

    # Histogram
    hist_orig = make_histogram_panel(original, size=(frame.shape[1], frame.shape[0]))
    hist_eq = make_histogram_panel(he_frame, size=(frame.shape[1], frame.shape[0]))

    # 2x2 Grid for global HE
    top_row = np.hstack((original, he_frame))
    bottom_row = np.hstack((hist_orig, hist_eq))
    grid = np.vstack((top_row, bottom_row))
    out.write(grid)

    # 2x1 CLAHE comparison
    comparison = np.hstack((he_frame, clahe_frame))
    clahe_out.write(comparison)

    gray_he = cv2.cvtColor(he_frame, cv2.COLOR_BGR2GRAY)
    gray_clahe = cv2.cvtColor(clahe_frame, cv2.COLOR_BGR2GRAY)
    contrast_he = rms_contrast(gray_he)
    contrast_clahe = rms_contrast(gray_clahe)
    print(f"RMS Contrast – HE: {contrast_he:.2f}, CLAHE: {contrast_clahe:.2f}")

/tmp/ipython-input-26-48157961.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  (bin_width*(j-1), size[1] - int(hist[j-1])),
/tmp/ipython-input-26-48157961.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  (bin_width*j, size[1] - int(hist[j])),


RMS Contrast – HE: 73.34, CLAHE: 56.46
RMS Contrast – HE: 73.35, CLAHE: 56.55
RMS Contrast – HE: 73.34, CLAHE: 56.62
RMS Contrast – HE: 73.37, CLAHE: 56.63
RMS Contrast – HE: 73.38, CLAHE: 56.71
RMS Contrast – HE: 73.40, CLAHE: 56.73
RMS Contrast – HE: 73.40, CLAHE: 56.56
RMS Contrast – HE: 73.42, CLAHE: 56.72
RMS Contrast – HE: 73.40, CLAHE: 56.72
RMS Contrast – HE: 73.37, CLAHE: 56.62
RMS Contrast – HE: 73.37, CLAHE: 56.63
RMS Contrast – HE: 73.36, CLAHE: 56.70
RMS Contrast – HE: 73.37, CLAHE: 56.74
RMS Contrast – HE: 73.38, CLAHE: 56.66
RMS Contrast – HE: 73.36, CLAHE: 56.70
RMS Contrast – HE: 73.35, CLAHE: 56.74
RMS Contrast – HE: 73.41, CLAHE: 56.75
RMS Contrast – HE: 73.38, CLAHE: 56.76
RMS Contrast – HE: 73.36, CLAHE: 56.61
RMS Contrast – HE: 73.37, CLAHE: 56.72
RMS Contrast – HE: 73.35, CLAHE: 56.74
RMS Contrast – HE: 73.37, CLAHE: 56.62
RMS Contrast – HE: 73.40, CLAHE: 56.62
RMS Contrast – HE: 73.36, CLAHE: 56.74
RMS Contrast – HE: 73.39, CLAHE: 56.70
RMS Contrast – HE: 73.35,

# **Release Resources**
Once the video is processed, the resources (video capture and output writers) are released.

In [ ]:
cap.release()
out.release()
clahe_out.release()

# **Re-encode .mp4 Videos to .mp4 Format and Display the Videos**

The .mp4 videos are re-encoded to ensure compatibility, and the processed videos are displayed in the notebook:


In [ ]:
os.system('ffmpeg -i outputyt_hist_eq_grid.mp4 outputyt_hist_eq_grid_final.mp4')
os.system('ffmpeg -i outputyt_clahe_comparison.mp4 outputyt_clahe_comparison_final.mp4')

display(Video("outputyt_hist_eq_grid_final.mp4", embed=True))
display(Video("outputyt_clahe_comparison_final.mp4", embed=True))

print("✅ Done! Global HE video: outputyt_hist_eq_grid_final.mp4")
print("✅ CLAHE comparison video: outputyt_clahe_comparison_final.mp4")

# This is the youtube video ccomparison grid video link

https://drive.google.com/file/d/1kPQtAk4eW155PBsRxv9ny1og7fhTQwWG/view?usp=sharing

# ***AFTER THIS PART WE WILL BE DOING THE RAW GITHUB VIDEO***

# **Download or Mount a Test Video**

This section checks whether the test video exists locally. If not, it downloads it from a URL.
VIDEO_URL: The URL to the test video.

VIDEO_PATH: The local file path where the video will be saved.

urllib.request.urlretrieve(): Downloads the video if it doesn't already exist on the local machine.

In [ ]:
VIDEO_URL = "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/vtest.avi"
VIDEO_PATH = "input.avi"

if not os.path.exists(VIDEO_PATH):
    urllib.request.urlretrieve(VIDEO_URL, VIDEO_PATH)

# **Open Video Streams**

The code uses OpenCV to open and extract details from the video, such as width, height, and frames per second (FPS).

cv2.VideoCapture(): Opens the video file.

cap.get(): Retrieves properties of the video, like frame dimensions and FPS.

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)


# **Initialize Video Output Writers**

Two video writers are initialized to save the processed videos:

1. **Global Histogram Equalization (HE) Output**: This writer saves the video after applying Global Histogram Equalization (HE) to enhance the overall contrast of the frames.
  
2. **CLAHE Output**: This writer saves the video after applying Contrast Limited Adaptive Histogram Equalization (CLAHE), which enhances the local contrast of the frames for better details in certain regions.


In [ ]:
# Output video writer for 2x2 grid (convert to mp4)
out = cv2.VideoWriter('output_hist_eq_grid.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps if fps > 0 else 24,
                      (frame_width * 2, frame_height * 2))

# Output writer for CLAHE comparison (convert to mp4)
clahe_out = cv2.VideoWriter('output_clahe_comparison.mp4',
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            fps if fps > 0 else 24,
                            (frame_width * 2, frame_height))



## **Histogram**
----
The histogram of gray level digital image with intensity values between $[0, L - 1]$ is a discrete function:

<font size="5">
$$h(r_k) = n_k$$
</font>

* $r_k$ : $k$-th intensity value
* $n_k$ : the number of pixels with $r_k$ value in the image




In [ ]:
def make_histogram_panel(bgr_frame, size=(320, 240), bins=64):
    panel = np.zeros((size[1], size[0], 3), dtype=np.uint8)
    colors = ('b', 'g', 'r')
    for i, color in enumerate(colors):
        hist = cv2.calcHist([bgr_frame], [i], None, [bins], [0, 256])
        cv2.normalize(hist, hist, alpha=0, beta=size[1], norm_type=cv2.NORM_MINMAX)
        bin_width = int(size[0] / bins)
        for j in range(1, bins):
            cv2.line(panel,
                     (bin_width*(j-1), size[1] - int(hist[j-1])),
                     (bin_width*j, size[1] - int(hist[j])),
                     color=(255 if color == 'b' else 0,
                            255 if color == 'g' else 0,
                            255 if color == 'r' else 0),
                     thickness=2)
    return panel

# **RMS Contrast Function**

The Root Mean Square (RMS) contrast function calculates the contrast of a grayscale image. It is defined as the standard deviation of pixel intensities.

The Root Mean Square (RMS) Contrast is a measure of image contrast, calculated as:

image.png

Where:

𝐼𝑖 is the pixel intensity at position
𝑖,

𝜇 is the mean pixel intensity,

𝑁 is the total number of pixels in the image.

This function calculates the square root of the average squared deviation from the mean of pixel values.

In [ ]:
def rms_contrast(gray_img):
    return np.sqrt(np.mean(np.square(gray_img - np.mean(gray_img))))

# **Main Processing Loop**

The code enters a loop where each video frame is processed. Two transformations are applied to each frame:

Global Histogram Equalization (HE): Improves the contrast of the image by adjusting the intensity distribution.

CLAHE (Contrast Limited Adaptive Histogram Equalization): A more advanced method for enhancing local contrast.
cv2.createCLAHE(): Initializes the CLAHE algorithm.

The loop reads each frame and applies both transformations.

# **Output Video Writing**

The processed frames are then written to the output video files:

2x2 Grid for Global HE: A grid combining the original frame and the histogram equalized frame, along with their histograms.

CLAHE Comparison: A side-by-side comparison of the global HE and CLAHE frames.

# **Displaying RMS Contrast**

The RMS contrast values for both the histogram equalized and CLAHE frames are calculated and printed.
This helps in comparing the contrast improvements between the two methods.

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original = frame.copy()

    # Global Histogram Equalisation
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    y_eq = cv2.equalizeHist(ycrcb[:, :, 0])
    ycrcb[:, :, 0] = y_eq
    he_frame = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    # CLAHE
    ycrcb_clahe = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    y_clahe = clahe.apply(ycrcb_clahe[:, :, 0])
    ycrcb_clahe[:, :, 0] = y_clahe
    clahe_frame = cv2.cvtColor(ycrcb_clahe, cv2.COLOR_YCrCb2BGR)

    # Histogram
    hist_orig = make_histogram_panel(original, size=(frame.shape[1], frame.shape[0]))
    hist_eq = make_histogram_panel(he_frame, size=(frame.shape[1], frame.shape[0]))

    # 2x2 Grid for global HE
    top_row = np.hstack((original, he_frame))
    bottom_row = np.hstack((hist_orig, hist_eq))
    grid = np.vstack((top_row, bottom_row))
    out.write(grid)

    # 2x1 CLAHE comparison
    comparison = np.hstack((he_frame, clahe_frame))
    clahe_out.write(comparison)

    gray_he = cv2.cvtColor(he_frame, cv2.COLOR_BGR2GRAY)
    gray_clahe = cv2.cvtColor(clahe_frame, cv2.COLOR_BGR2GRAY)
    contrast_he = rms_contrast(gray_he)
    contrast_clahe = rms_contrast(gray_clahe)
    print(f"RMS Contrast – HE: {contrast_he:.2f}, CLAHE: {contrast_clahe:.2f}")

# **Release Resources**

Once the processing is complete, the resources are released.
This ensures that the video streams and writers are properly closed.

In [ ]:
cap.release()
out.release()
clahe_out.release()

# **Re-encode .mp4 Videos to Standardized .mp4 Format and Display the Videos**

The .mp4 videos are re-encoded with H.264 video codec and AAC audio codec to ensure compatibility, and the processed videos are displayed in the notebook:


In [ ]:
os.system('ffmpeg -i output_hist_eq_grid.mp4 -c:v libx264 -c:a aac -strict experimental output_hist_eq_grid_final.mp4')
os.system('ffmpeg -i output_clahe_comparison.mp4 -c:v libx264 -c:a aac -strict experimental output_clahe_comparison_final.mp4')

display(Video("output_hist_eq_grid_final.mp4", embed=True))
display(Video("output_clahe_comparison_final.mp4", embed=True))

print("✅ Done! Global HE video: output_hist_eq_grid_final.mp4")
print("✅ CLAHE comparison video: output_clahe_comparison_final.mp4")

# This is the github video ccomparison grid video link

https://drive.google.com/file/d/1qk8wfkhAmxWQNv6cQE3KwajnkgRGbSnZ/view?usp=sharing